In [1]:
import pickle
a = open("Articles.pickle",'rb')
uncleanedarticles = pickle.load(a)
articles=[]
text=""
title=""
titles=[]
sources=[]
for i in uncleanedarticles:
    source,url,title,text = i[0],i[1],i[2],i[3]
    sources.append(source)
    text = text.replace('\n\n','\n')
    if text =="":
        continue
#     article = i.split('\n\n\n\n\n')
#     text = article[1].strip('\n').strip(' ')
    if text.count('\n') <10:
        continue
    if text.count('https://') >10:
        continue   
#     title = article[0].strip('\n').strip(' ')
    title = title.strip(' ')
    if title in titles:
        continue
    titles.append(title)
    articles.append([source,url,title,text])
a.close()
# print(articles)

FileNotFoundError: [Errno 2] No such file or directory: 'Articles.pickle'

In [2]:
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
ps = SnowballStemmer('english')
wordRepArticles =[]
idf={}

articleslen=[]
for i in articles:
    title,text = i[2],i[3]
    text = text.replace('\\','');text = text.replace('.\n','\n');text = text.replace('\n\n','\n')
    articledist={}
    sentencelis=[]
    count=0
    for i in (text+'\n'+title).split('\n'):
        if i ==title:
            mult =2
        else:
            mult =1
        linedist={}
        v = i.split(' ')
        if len(v)<4:
            continue
        for word in v:
            if word not in stopwords.words('english'):
                word = word.replace('(','');word = word.replace(')','')
                word = word.replace('"','');word = word.replace(',','');word = word.strip(' ')
                stemmed = ps.stem(word)
                if stemmed in articledist.keys():
                    articledist[stemmed]+=mult
                else:
                    articledist[stemmed]=mult
                if stemmed in linedist.keys():
                    linedist[stemmed]+=1
                else:
                    linedist[stemmed]=1
            count+=1
        sentencelis.append([i,linedist])
    for words in articledist.keys():   
        if words in idf.keys():
            idf[words]+=1
        else:
            idf[words]=1
    wordRepArticles.append([title,articledist,sentencelis])
    articleslen.append(count)

NameError: name 'articles' is not defined

In [ ]:
# print(len(wordRepArticles))
# print(wordRepArticles[0])
print(len(articles))
print(len(uncleanedarticles))
fileObject = open('./mysite/cleanedarticles.pickle','wb')
pickle.dump(articles,fileObject)
fileObject.close()

In [ ]:
articles[:3]

In [ ]:
import math
n = len(articles)
avgArticleLen = sum(articleslen)*1.0/n
match = [[0 for i in range(n)] for i in range(n)]
for i in range(n):
    mainarticle = wordRepArticles[i]
    mainlength = articleslen[i]
    for j in range(i+1,n):
        comparearticle = wordRepArticles[j]
        comparelength = articleslen[j]
        commonwords = list(set(mainarticle[1].keys()) & set(comparearticle[1].keys())) 
        value = 0
        for word in commonwords:
            mainvalue = (1+math.log(mainarticle[1][word]))*1.0/(math.log(1+mainlength/avgArticleLen))
            compvalue = (1+math.log(comparearticle[1][word]))*1.0/(math.log(1+comparelength/avgArticleLen))
            value += mainvalue*compvalue* math.log(1+n/idf[word])
        match[i][j] = value
        match[j][i] = value

In [ ]:
topmatches = [[] for i in range(n)]
matches = 3
for i in range(n):
    vals = [[match[i][j],j] for j in range(n)]
    toparticle = sorted(vals,reverse=True)[:matches]
    print("Main title: "+ articles[i][2])
    for j in range(matches):
        topmatches[i].append(toparticle[j])
        print(str(j)+"/" + str(matches)+ ": " + articles[toparticle[j][1]][2])

In [ ]:
print(avgArticleLen)

In [ ]:
sentmatch = []
for i in range(n):
    mainarticle = wordRepArticles[i]
    rep=[]
    for j in range(matches):
        comparearticle = wordRepArticles[topmatches[i][j][1]]
        bestmatchSentence =[]
        for k in range(len(mainarticle[2])):
            val = -1
            msentence = mainarticle[2][k][1]
            mainlength = len(mainarticle[2][k][0].split(' '))
            for l in range(len(comparearticle[2])):
                csentence = comparearticle[2][l][1]
                comparelength = len(comparearticle[2][l][0].split(' '))
                commonwords = list(set(msentence.keys()) & set(csentence.keys())) 
                value = 0
                for word in commonwords:
                    mainvalue = (1+math.log(msentence[word]))*1.0/(math.log(1+mainlength/avgArticleLen))
                    compvalue = (1+math.log(csentence[word]))*1.0/(math.log(1+comparelength/avgArticleLen))
                    value += mainvalue*compvalue* math.log(1+n/idf[word])
                if value>val:
                    val = value
                    ind = l
            bestmatchSentence.append([ind,val,comparearticle[2][ind][0]])
        rep.append(bestmatchSentence)
    sentmatch.append(rep)

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer() 

reliability = [9.9 for i in range(len(sources))]
articleaggreeement=[]
for i in range(n):
    articleval =0
    mainarticle = wordRepArticles[i]
    for j in range(matches):
        val=0
        comparticle=wordRepArticles[topmatches[i][j][1]]
        for k in range(len(mainarticle[2])):
            sent1 = analyzer.polarity_scores(mainarticle[2][k][0])['compound']
            sent2 = analyzer.polarity_scores(sentmatch[i][j][k][2])['compound']
            val +=sent1*sent2
        val /= len(mainarticle[2])
        articleval +=val*reliability[sources.index(articles[topmatches[i][j][1]][0])]
    articleaggreeement.append(articleval/matches)
    articles[i].append(articleval/matches)
    articles[i].append(i)

In [ ]:
# fileObject = open('cleanedarticles.pickle','wb')
# pickle.dump(articles,fileObject)
# fileObject.close()
print(articles[:2])

In [ ]:
sourcerelia=[]
for i in range(len(sources)):
    sourcerelia.append([sources[i],reliability[i]])
fileObject = open('./mysite/sourcereliability.pickle','wb')
pickle.dump(sourcerelia,fileObject)
fileObject.close()
# print(sourcerelia[0])

In [ ]:
articlesimilarity=[]
for i in range(n):
    for j in range(len(topmatches[i])):
#         articlesimilarity.append([articles[i][2],articles[topmatches[i][j][1]][2]])
        articlesimilarity.append([i,topmatches[i][j][1],topmatches[i][j][0]])
fileObject = open('./mysite/articlessimilarity.pickle','wb')
pickle.dump(articlesimilarity,fileObject)
fileObject.close()

In [ ]:
print(articlesimilarity[0])

In [ ]:
# fileObject = open('./mysite/articlessimilarity.pickle','rb')
# a=pickle.load(fileObject)
# fileObject.close()

In [ ]:
# a[0]